In [97]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import json
import geopandas as gpd
import folium
from shapely.geometry import Point
from statsmodels.formula.api import glm
import statsmodels.api as sm 
import numpy as np

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [98]:
endpoint = ('https://data.nashville.gov/resource/2u6v-ujjs.json?$limit=50000')
parameters = {
    '$where': 'incident_reported between "2021-01-01T00:00:00" and "2021-06-30T23:59:59"',
    'offense_description' : "BURGLARY- AGGRAVATED"
}
response = requests.get(endpoint, params = parameters)

In [99]:
burglaries = (pd.read_json(response.text)
            .drop_duplicates(subset = "incident_number"))

2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [100]:
census_tracts = gpd.read_file("./tl_2019_47_tract/tl_2019_47_tract.shp")

In [101]:
burglaries['geometry'] = burglaries.apply(lambda x: Point((x.longitude,   #creating geometry column
                                                       x.latitude)),
                                      axis=1)

geo_burglaries = gpd.GeoDataFrame(burglaries, 
                                crs = census_tracts.crs, 
                                geometry = burglaries['geometry'])

In [102]:
#clean up census and burglaries_geo

census_tracts = census_tracts[['COUNTYFP','NAMELSAD', 'TRACTCE', 'geometry']]
geo_burglaries = geo_burglaries[['incident_location','geometry']]

In [103]:
geo_burg_census = gpd.sjoin(geo_burglaries, census_tracts, op = 'within')

In [104]:
geo_burg_census.NAMELSAD.mode() #Census Tract 163 had highest number of aggravated burglaries 

0    Census Tract 163
dtype: object